In [1]:
import os
import cv2
import imutils
import warnings
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from tqdm import tqdm
from scipy.spatial import distance

In [ ]:
#Turn a frequency dataframe into a TF-IDF dataframe
def TF_IDF(data,start_col):
    try :
        #Charger csv, enlever colonnes inutiles
        TF= data.iloc[:,start_col:]
        
        #Préparer l'IDF, donnant la quantité d'information apportée par un tag en fonction de sa rareté
        IDF = pd.DataFrame(data = np.zeros(len(TF.columns)),index = TF.columns)
        for i in tqdm(range(0,len(IDF))) :
          IDF.iloc[i,0] = np.log(len(TF)/TF.iloc[:,i].sum())
        
        #TD_IDF = exprimer chaque image en un vecteur produit de la quantité d'info des tags et des tags
        TF_IDF = pd.DataFrame(index= data["url"], columns= IDF.index)
        for i in tqdm(range(0,len(TF_IDF))) :
          TF_IDF.iloc[i,:] = TF.iloc[i,:] * IDF.iloc[:,0]
        
       
        #Sauvegarder TF IDF
        TF_IDF.to_csv("./Data/TF_IDF.csv")
        return TF_IDF
    except KeyboardInterrupt as stop :
        TF_IDF.to_csv("./Data/TF_IDF.csv")
        return TF_IDF

In [ ]:
#Compute cosine distance of a dataframe
def cos_distance(data) :
    try :
        #Calcul distances cosine
        distances_matrix = pd.DataFrame(index = data.index, columns = data.index)
        
        for i in tqdm(range(0,len(distances_matrix))):
            for j in range(i,len(distances_matrix)) :
                if i == j :
                    distances_matrix.iloc[i,j] = 0
                else :
                    distances_matrix.iloc[i,j] = distance.cosine(np.array(data.iloc[i,:]).astype(float),
                                                                  np.array(data.iloc[j,:]).astype(float))
                    distances_matrix.iloc[j,i] = distances_matrix.iloc[i,j]
        distances_matrix.to_csv("./Data/cos_distances.csv") 
        return distances_matrix
    except KeyboardInterrupt as stop :
        return distances_matrix
        distances_matrix.to_csv("./Data/cos_distances.csv")

In [ ]:
#Chi square distance
def chi2_distance(image1, image2):
    with warnings.catch_warnings():
        warnings.filterwarnings('error')
        chi = 0
        for (a, b) in zip(image1, image2) :
            try :
                chi += ((a - b) ** 2) / (a + b)
            except Warning as e:
                #Used to solve problems when a = b = 0
                pass
        chi = 0.5* chi
        return chi

In [ ]:
#https://www.pyimagesearch.com/2014/12/01/complete-guide-building-image-search-engine-python-opencv/
#Classed use to extract color histograms from images
class ColorDescriptor:
    def __init__(self, bins):# store the number of bins for the 3D histograms
        self.bins = bins
    def describe(self, image):
        # convert the image to the HSV color space and initialize
        # the features used to quantify the image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        features = []
        # grab the dimensions and compute the center of the image
        (h, w) = image.shape[:2]
        (cX, cY) = (int(w * 0.5), int(h * 0.5))
        
        # divide the image into four rectangles/segments (top-left,
        # top-right, bottom-right, bottom-left)
        segments = [(0, cX, 0, cY), (cX, w, 0, cY), (cX, w, cY, h),
                    (0, cX, cY, h)]
        # construct an elliptical mask representing the center of the
        # image
        (axesX, axesY) = (int(w * 0.75) // 2, int(h * 0.75) // 2)
        ellipMask = np.zeros(image.shape[:2], dtype = "uint8")
        cv2.ellipse(ellipMask, (cX, cY), (axesX, axesY), 0, 0, 360, 255, -1)
        # loop over the segments
        for (startX, endX, startY, endY) in segments:
            # construct a mask for each corner of the image, subtracting
            # the elliptical center from it
            cornerMask = np.zeros(image.shape[:2], dtype = "uint8")
            cv2.rectangle(cornerMask, (startX, startY), (endX, endY), 255, -1)
            cornerMask = cv2.subtract(cornerMask, ellipMask)
            # extract a color histogram from the image, then update the
            # feature vector
            hist = self.histogram(image, cornerMask)
            features.extend(hist)
        # extract a color histogram from the elliptical region and
        # update the feature vector
        hist = self.histogram(image, ellipMask)
        features.extend(hist)
        # return the feature vector
        return features
    
    def histogram(self, image, mask):
        # extract a 3D color histogram from the masked region of the
        # image, using the supplied number of bins per channel
        hist = cv2.calcHist([image], [0, 1, 2], mask, self.bins,
                            [0, 180, 0, 256, 0, 256])
        # normalize the histogram if we are using OpenCV 2.4
        if imutils.is_cv2():
            hist = cv2.normalize(hist).flatten()
            # otherwise handle for OpenCV 3+
        else:
            hist = cv2.normalize(hist, hist).flatten()
            # return the histogram
        return hist